In [1]:
import splinepy
import vedo
from sdf_microstructure import (
    create_microstructure_from_experiment,
    export_mesh,
    tetrahedralize_surface,
)

from deep_sdf.analysis import linear_elasticity as le

vedo.settings.default_backend = "k3d"

test_mesh = True

In [2]:
faces, jacobian = create_microstructure_from_experiment("/home/michael.kofler/DeepSDF/experiments/round_cross_big_network", tiling=[4,2,2], N_base=20)

In [3]:
if test_mesh:
    rect = splinepy.helpme.create.box(2, 1, 1)
    volumes = rect.extract.volumes(resolution=30)
else:
    volumes = tetrahedralize_surface(faces)

In [4]:
export_mesh(volumes, "test_mesh.mesh", show_mesh=False)
le_problem = le.LinearElasticityProblem()
le_problem.read_mesh("test_mesh.mesh")
le_problem.show_mesh()

K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…

In [5]:
export_mesh(volumes, "test_mesh.mesh")
le_problem = le.LinearElasticityProblem()
le_problem.read_mesh("test_mesh.mesh")


le_problem.solve(ref_levels=0)
# le_problem.compute_compliance()


Number of state unknowns: 81000
Number of filter unknowns: 27000
Number of control unknowns: 24389
Finished Solution. Max deflection: 0.0127065425764567


In [56]:
print(le_problem.compute_volume())

(12383.262333209626, None)


In [7]:
le_problem.compute_compliance()

In [8]:
print(le_problem.compute_volume())

(nan, None)


In [9]:
le_problem.show_solution(output="u_vec")

K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…

In [10]:
le_problem.show_solution(output="strain_energy_density")

K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…